## Convert MU69 halo surface brightness into particle density. 

In theory I could do this theoretically. There should be one published correct value for (mag per arcsec2, at a given distance) to (surface albedo). Assuming zero phase.

I mean, mags per arcsec2 -- that's should translate directly into an amount of flux.

Maybe just bootstrap off of the sun: -26.7 = 1400 W/m2. That's probably the best.

## Intialize things

In [2]:
import numpy as np
import astropy
import astropy.units as u
import astropy.constants as c
import math

In [56]:
# Define handy functions for magnitude conversion

def fac2mag(fac):
    import numpy as np
    return np.log( 1/(1.*fac))/np.log(100**0.2)    

def mag2fac(mag): 
    import numpy as np
    return np.exp(-mag * np.log(100.**0.2))

In [4]:
# Define constants and inputs

mag_sun  = -26.7
dist_kbo = 43.4*u.AU  # Not sure of the current distance. Orbit is circular.: 42 .. 46 AU.
pi       = math.pi    # Just for convenience
dx       = dist_kbo * 1*u.arcsecond.to('radian') # Calc the dim's of a 1 arcsec 
                                                 # paper target as seen from Sun
    
N_desired  = 0.001                                # Number of hits 
N_required = 0.01                               # Number of hits 

mpas_kbo_hst = 28            # Mags per arcsec. Alex Parker limit
albedo_kbo   = 0.2           # Albedo assumed for the dust. 
r_dust       = 50*u.micron   # Size of the dust. A bit smaller than the sc safety limit.
#r_dust       = 0.2*u.mm      # Size of the dust, in my whitepaper
A_sc         = 5*(u.meter)**2 # S/C cross-section, in my whitepaper

Let's put a 1 arcsec piece of white paper at 43 AU. How much solar flux does it reflect back to Earth?

In [5]:
L_paper = c.L_sun * 1/(4*pi*(dist_kbo)**2) * dx**2 * 1/(4*pi*(dist_kbo)**2)

How big is that paper?

In [6]:
dx.to('km')

<Quantity 31476.75896101348 km>

In [7]:
L_paper.to('W/m**2')

<Quantity 1.3580288693349867e-12 W / m2>

Validate that my solar flux is correct

In [8]:
print("Solar flux = {:.2f}".format((c.L_sun / (4*pi*(1*u.AU)**2)).to('W/m2')))

Solar flux = 1367.57 W / m2


Now, how many magnitudes per arcsec is this? Sun has a total brightness of -26.7, as seen on Earth. How many times fainter is this white piece of paper, than the Sun would be, at Earth?

In [9]:
ratio = L_paper / (c.L_sun / (4*pi*(1*u.AU)**2))

And now print the magnitude of this paper. Since it is exactly 1" across, this **is** the number of mags per arcsec already.

In [10]:
mag_paper  = mag_sun + fac2mag(ratio)  # Mag of white paper, seen from Earth
mpas_paper = mag_paper
print("Mags/arcsec of white paper at 43 AU = {}".format(mpas_paper))

Mags/arcsec of white paper at 43 AU = 10.807598973500095


Now, we know the mags per arcsec of the region around MU69, as measured by Alex Parker. That means we can just calculate the 'filling factor' -- that is, how much fainter is that region, than a 1" piece of paper.

In [11]:
fillfac = mag2fac(mpas_kbo_hst - mpas_paper)
print ("Geometric filling factor (ie, tau) = {:.2e}".format(fillfac))

Geometric filling factor (ie, tau) = 1.33e-07


Ignore this 'Filling Factor' stuff. We want to do this in terms of I/F.

Now convert to particles per square meter, and bring in albedo.

In [12]:
n = fillfac * ((1*u.meter**2) / (pi * r_dust**2)).to('1') / albedo_kbo # Number density per sq meter
N = n * A_sc.to('m2').value                                            # Number of particles hit during encounter

## Print the final results

In [13]:
print("{:.2f} particles of radius {} per square meter".format(n, r_dust))
print("{:.2f} particles hitting s/c during encounter".format(N))

84.51 particles of radius 50.0 micron per square meter
422.56 particles hitting s/c during encounter


Now, let's say we want to do this backwards: for a given N, what fillfac do we want? It is just linear scaling.

In [14]:
N   = N_required / A_sc
N / 84

<Quantity 2.380952380952381e-05 1 / m2>

NB: Becuase this is in reflected light (not transmitted) we want to deal in I/F, not tau.

In [15]:
84 / 0.05

1680.0

# Part 2: Do some estimates for how a sunflower dust ring would affect the rotational light curve

Alex Parker says that MU69 rotational light curve is 0.2 mags. This is pretty normal for KBOs, according to Trilling & Bernstein (2006), who list light curves for 65 of them, and nearly all are < 0.5, with many < 0.1. However, with MU69 being a binary / oblong, it is harder to explain why the magnitude is so small.

Hypothesis: if surface area of ring is comparable to surface area of MU69, then light curve could be affected.

In [57]:
r_mu69    = 10*u.km
r_annulus = np.array([4000, 6000])*u.km # This is the annulus that AP searched.

In [58]:
A_annulus = pi * (r_annulus[1]**2 - r_annulus[0]**2)
A_mu69 = pi * r_mu69**2

In [59]:
ratio = A_annulus * fillfac / A_mu69

In [60]:
ratio

<Quantity 0.026550309316382137>

Well, there's our answer. With the *measured upper limit* amount of dust, the amount of surface area in the ring in no way approaches the surface area of MU69.

With the *dangerous* amount of dust -- which is much much less -- then the ring surface area would be that much smaller even.